In [ ]:
import ipywidgets as w
import nglview as nv
import glob
import os
from gmx import GMX
import re
import time
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md

In [ ]:
%matplotlib widget

In [29]:
cores=4
gpus=1

In [ ]:
pdbview = nv.NGLWidget()
pdbview_component = None
cartoon = w.Checkbox(description='Cartoon',value=True)
licorice = w.Checkbox(description='Licorice',value=True)

def on_checkbox_click(e):
    set_pdbview_representation()

cartoon.observe(on_checkbox_click,names='value')
licorice.observe(on_checkbox_click,names='value')

In [ ]:
pdbs = w.Dropdown(
    options=glob.glob('*.pdb') + ['none'],
    value='none',
    description='Current filename',
    disabled=False,
)

def set_pdbview_representation():
    pdbview.clear()
    if cartoon.value:
        pdbview.add_cartoon(color='residueno')
    if licorice.value:
        pdbview.add_licorice()


def pdb_choose(e):
    global pdbview_component
    if pdbview.n_components > 0:
        pdbview.remove_component(pdbview_component)
    pdbs.value = e['new']
    if pdbs.value != 'none':
        pdbview_component = pdbview.add_component(pdbs.value)
        set_pdbview_representation()
    
pdbs.observe(pdb_choose,'value')

In [ ]:
upload = w.FileUpload(accept='pdb',multiple=False)

def on_upload_pdb(c):
    fname = list(upload.value.keys())[0]
    with open(fname,'wb') as f:
        f.write(list(upload.value.values())[0]['content'])
    
    pdbs.options = glob.glob('*.pdb') + ['none']
    upload.value.clear()
    pdb_choose({'new':fname})
       
upload.observe(on_upload_pdb,'_counter')

In [ ]:
start = w.Button(
    description='Run',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run',
    icon='check', # (FontAwesome names without the `fa-` prefix)
)

In [ ]:
pprep = w.FloatProgress(
    value=0.,min=0.,max=1.,
    description='Idle',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'blue'},
    orientation='horizontal'
)


err = w.Textarea(description='Error:',layout=w.Layout(width='90%',visibility='hidden'))

In [ ]:
prepfig,prepax = plt.subplots()
prepfig.canvas.layout.visibility = 'hidden'
prepfig.canvas.layout.width='80%'
prepfig.canvas.layout.height='10%'

In [ ]:
prepare = w.VBox([err, w.HBox([start, pprep]),prepfig.canvas])


In [ ]:
mnt=os.popen('mount | grep /home/jovyan').read()
pvcid=re.search('pvc-[0-9a-z-]+',mnt).group(0)
pvc=os.popen(f'kubectl get pvc | grep {pvcid} | cut -f1 -d" "').read().rstrip()
pvc

In [ ]:
gmx = GMX(pvc=pvc,workdir='')

In [ ]:
def check_gmx_stat(gmx,bar,step,err):
    while not gmx.status().succeeded and not gmx.status().failed:
        time.sleep(1)
    if gmx.status().succeeded:
        if bar: bar.value += step
        ret = True
    else:
        err.value = gmx.log()
        err.layout.visibility = 'visible'
        if bar:
            bar.style.bar_color = 'red'
            bar.description = 'Error'
        ret = False
        
    return ret

In [ ]:
def watch_gmx(gmx,logf,imaxs,bar,err):
    maxs = imaxs
    err.layout.visibility = 'hidden'
    err.value = ''
    bar.style.bar_color = 'blue'
    bar.value = 0.
    while not gmx.status().succeeded and not gmx.status().failed:
        try:
            with open(logf) as log:
                lines = log.readlines()
                for l in reversed(lines):
                    if re.match('\s+Step\s+Time',l):
                        s = float(re.match('\s+(\d+)\s+',prev).group(1))
                        break
                    prev = l
            if s > maxs:
                maxs += imaxs
            bar.value = s / maxs
        except FileNotFoundError:
            pass
        
        time.sleep(1)
        
    if gmx.status().succeeded:
        bar.style.bar_color = 'green'
        return True
    else:
        bar.style.bar_color = 'red'
        err.layout.visibility = 'visible'
        err.value = gmx.log()
        bar.description = 'Error'
        return False
        



In [ ]:
        
pprep.value = 0
err.value = ''
mdbox=2.0

def rmf(file):
    try:
        os.unlink(file)
    except FileNotFoundError:
        pass

def run_prepare(e):
    pdb = pdbs.value
    base = pdb.replace('.pdb','')
    pprep.description = 'Solvating'
    err.layout.visibility = 'hidden'
    err.value = ''
    ok = True
    for cmd in [
        f"pdb2gmx -f {pdb} -o {base}.gro -p {base}.top -water tip3p -ff amber94 -ignh",
        f"editconf -f {base}.gro -o {base}-box.gro -c -d {mdbox} -bt dodecahedron",
        f"solvate -cp {base}-box.gro -cs spc216.gro -o {base}-solv.gro -p {base}.top",
        f"grompp -f ions.mdp -c {base}-solv.gro -p {base}.top -o ions.tpr",
        (f"genion -s ions.tpr -o {base}-ions.gro -p {base}.top -pname NA -nname CL -neutral","13"),
     ]:
        if isinstance(cmd,str):
            gmx.start(cmd)
        else:
            gmx.start(cmd[0],input=cmd[1])
        ok = check_gmx_stat(gmx,pprep,.2,err)
        gmx.delete()
        if not ok: return

    if ok:
        pprep.style.bar_color = 'green'
        
    pprep.description = 'Minimizing'
    pprep.value = 0.
    gmx.start(f"grompp -f minim-sol.mdp -c {base}-ions.gro -p {base}.top -o em.tpr")
    ok = check_gmx_stat(gmx,None,0.,err)
    gmx.delete()
    if not ok: return
    rmf('em.log')
    gmx.start(f"mdrun -v -deffnm em -pin on",cores=cores,gpus=gpus)
    ok = watch_gmx(gmx,'em.log',500,pprep,err)
    gmx.delete()
    if not ok: return
    
    pprep.description = 'NVT'
    pprep.value = 0.
    gmx.start(f"grompp -f nvt.mdp -c em.gro -r em.gro -p {base}.top -o nvt.tpr")
    ok = check_gmx_stat(gmx,None,0,err)
    gmx.delete()
    if not ok: return
    rmf('nvt.log')
    gmx.start(f"mdrun -pin on -deffnm nvt",cores=cores,gpus=gpus)
    ok = watch_gmx(gmx,'nvt.log',50000,pprep,err)
    gmx.delete()
    if not ok: return
    
    pprep.description = 'NPT'
    pprep.value = 0.
    gmx.start(f"grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p {base}.top -o npt.tpr")
    ok = check_gmx_stat(gmx,None,0,err)
    gmx.delete()
    if not ok: return
    rmf('npt.log')
    gmx.start(f"mdrun -pin on -deffnm npt",cores=cores,gpus=gpus)
    ok = watch_gmx(gmx,'npt.log',50000,pprep,err)
    gmx.delete()
    if not ok: return
    
    
def prep_graph():
    pprep.description = 'Generating graph'
    pprep.value = 0

    """for cmd in [
        (f"energy -f npt.edr -o press.xvg",18),
        (f"energy -f npt.edr -o dens.xvg",24),
        (f"energy -f npt.edr -o temp.xvg",16)
    ]:
        gmx.start(cmd[0],input=cmd[1])
        ok = check_gmx_stat(gmx,pprep,1./3.,err)
        gmx.delete()
        if not ok:            
            break
"""
        
    temp = np.loadtxt('temp.xvg',comments=['#','@'])
    press = np.loadtxt('press.xvg',comments=['#','@'])
    dens = np.loadtxt('dens.xvg',comments=['#','@'])

    prepax.clear()

#    prepfig.title.value = 'isothermal-isobaric equilibration'
    prepax.grid()
    #plt.xlabel('time (ps)')
    #prepax.ylabel("pressure (bar)")

    prepax.plot(dens[:,0],dens[:,1],label='density',color='r')
    prepax.set_ylabel('density',color='r')
    prepax.tick_params(axis='y', labelcolor='r')
    
    ax2 = prepax.twinx()
    ax2.plot(press[:,0],press[:,1],label='press',color='g')
    ax2.set_ylabel('press (bar)',color='g')
    ax2.tick_params(axis='y', labelcolor='g')
    
    ax3 = prepax.twinx()
    ax3.plot(temp[:,0],temp[:,1],label='temperature',color='b')
    ax3.set_ylabel('temperature (K)',color='b')
    ax3.tick_params(axis='y', labelcolor='b')

    prepfig.canvas.layout.visibility = 'visible'
    prepfig.canvas.draw()
    prepfig.canvas.flush_events()
    
    
start.on_click(run_prepare)

In [ ]:
bias = w.Label('TODO')

In [ ]:
nsec = w.FloatText(
    value=5.,
    description='Simulation length (ns):',
    disabled=False
)

mdprog = w.FloatProgress(
    value=0.,min=0.,max=1.,
    description='Progress:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'blue'},
    orientation='horizontal'
)

mdstart = w.Button(
    description='Run',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run',
    icon='check', # (FontAwesome names without the `fa-` prefix)
)

mderr = w.Textarea(description='Error:',layout=w.Layout(width='90%',visibility='hidden'))

def run_md(e):
    base = pdbs.value.replace('.pdb','')
    mdprog.value = .05
    nsteps = int(500 * 1000 * nsec.value) # XXX: hardcoded dt = 2fs
    !cp md.mdp.template md.mdp
    with open('md.mdp','a') as mdp:
        mdp.write(f"nsteps = {nsteps}\n")   
        
    gmx.start(f"grompp -f md.mdp -c npt.gro -t npt.cpt -p {base}.top -o md.tpr")
    ok = check_gmx_stat(gmx,None,0,mderr)
    gmx.delete()
    if not ok: return
    gmx.start(f"mdrun -pin on -deffnm md",cores=cores,gpus=gpus)
    ok = watch_gmx(gmx,'md.log',nsteps,mdprog,mderr)
    gmx.delete()
    if not ok: return
    


mdstart.on_click(run_md)   


In [ ]:
# -i 1 trjconv -f {xtc} -s npt.gro -pbc nojump -o {pbc}
# 

def view_xtc(e):
    global pdbview_component
    base = pdbs.value.replace('.pdb','')
    gmx.start(f"trjconv -f md.xtc -s npt.gro -pbc nojump -o pbc.xtc",input=1)
    check_gmx_stat(gmx,None,0,mderr)
    gmx.delete()
    
    tr = md.load_xtc('pbc.xtc',top=base + '.gro')
    idx=tr[0].top.select("name CA")
    tr.superpose(tr[0],atom_indices=idx)
    if pdbview_component:
        pdbview.remove_component(pdbview_component)
    pebview_component = pdbview.add_trajectory(tr)
    
trbutton = w.Button(description='Reload trajectory',disabled=False,button_style = '')
trbutton.on_click(view_xtc)


In [ ]:
mdrun = w.VBox([nsec,mdprog,mderr,mdstart,trbutton])

In [ ]:
def switch_view(e):
    global pdbview_component
    if pdbview_component:
        pdbview.remove_component(pdbview_component)
        pdbview_component = None
    if e.new == 0:
        pdbview_component = pdbview.add_component(pdbs.value)
        set_pdbview_representation()
    elif e.new == 2:
        tr = md.load_xtc('pbc.xtc',top='mol' + '.gro')
        idx=tr[0].top.select("name CA")
        tr.superpose(tr[0],atom_indices=idx)
        pdbview_component = pdbview.add_trajectory(tr)

In [ ]:
tabs = w.Tab()
tabs.children = [prepare, bias, mdrun]
tabs.set_title(0,'Prepare molecule')
tabs.set_title(1,'Bias potential')
tabs.set_title(2,'Run MD')

tabs.observe(switch_view,'selected_index')

main = w.VBox([w.HBox([upload, pdbs]), w.HBox([pdbview, w.VBox([cartoon, licorice])]), tabs])
display(main)
